# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa
import binascii
import hashlib


# # Generate a private key
private_key_raw_object = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
print("Private key raw:", private_key_raw_object.to_string())

private_key_raw = private_key_raw_object.to_string()

# Get the private key
# private_key_hex = private_key_raw.to_string().hex()
# private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_raw, curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key:", private_key.to_string().hex())

# Get the public key
# public_key: ecdsa.VerifyingKey = private_key.get_verifying_key()
public_key = private_key.get_verifying_key()

# Print the public key in hexadecimal format
print(f"Public key: {public_key.to_string("compressed").hex()}")
# print(f"Public key: {public_key.hex()}")

# def compress_public_key(public_key: bytes) -> bytes:
#     x = public_key[:32]
#     y = public_key[32:]
#     if int.from_bytes(y, 'big') % 2 == 0:
#         return b'\x02' + x
#     else:
#         return b'\x03' + x
    
# Print the compressed public key
# print(f"Compressed public key: {compress_public_key(public_key.to_string('compressed')).hex()}")
# print(f"Compressed public key: {public_key.to_string('compressed').hex()}")

# # Convert private key to hex
# private_key = private_key_raw.to_string().hex() + '01'
# print("Private key:", private_key)

# # Generate a public key
# public_key = private_key_raw.get_verifying_key()
# print("Public key: ", public_key.to_string().hex())

Private key raw: b'\x97O\xae-by[\xf3\xfd\xb6X\xf2\x1b\xcdH\xc2As\x19\x8417e\xd5\xf3\x89$U7\xb1\x7fN'
Private key: 974fae2d62795bf3fdb658f21bcd48c241731984313765d5f389245537b17f4e
Public key: 02b2465d9ebc12e4c856a1347b4df66114eaa717aee3b388e41d520531c851dc13


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256: bytes = hashlib.sha256(public_key.to_string('compressed')).digest()
print("SHA256: ", sha256.hex())

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()
print("RIPEMD-160: ", hashed_public_key.hex())

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]
print("Address checksum: ", address_checksum.hex())

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum
print("Binary Bitcoin Address: ", binary_bitcoin_address.hex())

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address).decode()
print("Bitcoin address: ", bitcoin_address)

SHA256:  42e92287621767e53b8b280f1c8c7b9e4a7cca29d5e27357b6a0cd497e51bdfb
RIPEMD-160:  c1e560c56998a850b46a4d13544dc6047fc3fc8c
Address checksum:  4d3f7168
Binary Bitcoin Address:  00c1e560c56998a850b46a4d13544dc6047fc3fc8c4d3f7168
Bitcoin address:  1JgECVF9zHQKUBRRD9njbFurhCCBXRi9MR


### Verify a bitcoin address

In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [5]:
# Path: concepts_bitcoin_transactions.ipynb

private_key_raw_1 = private_key_raw_object

# Use the private key to sign a message
# message = bytes("Bitcoin Signed Message:\nHello world!", "utf-8")
message = b"Bitcoin Signed Message:\n" + b"Hello world!"
signature = private_key.sign(message, hashfunc=hashlib.sha256)
print("Signature:", signature.hex())
print("Signature:", binascii.hexlify(signature).decode())


Signature: 00c7d170e3f82f315a18ca9357b8f42866fb12ec65a9c932932b10714182a85465e4ec9c3ed7f167ae20caa4f444390344a3fe701334f284b1a13f30167cd2de
Signature: 00c7d170e3f82f315a18ca9357b8f42866fb12ec65a9c932932b10714182a85465e4ec9c3ed7f167ae20caa4f444390344a3fe701334f284b1a13f30167cd2de


### Verify a message

In [6]:
# Use the public key to verify a message
print(public_key.verify(signature, message))

True


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [7]:
# pip install pybitcoin

In [8]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [9]:
# pip install bitcoin-utils

In [10]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [11]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [12]:
# pip install python-bitcoinlib

In [13]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress, CBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_)
# bitcoin_address_ = CBitcoinAddress(bitcoin_address_)
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_msg.magic} {bitcoin_msg.message} {bitcoin_msg.GetHash()}")

signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.hex()}")

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: L5Ws9bzCFdqXCs99jqzaNvmchHjY5xmeMCzsXX7ZXdwnEofjGezc
Public key: 03aa1b69f81f9fcad8b5415a5551236fe2c6fbdc92a4e92dc2372afd649e9f1c06
Bitcoin address: 1FNCiUkMPYqCFKncLPuajAJoQavVh9182h
Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' b'\x9bE\xfc\xc9\x9e\xd8JpQ1|\xc9\xc8\x04\xb0#Y%\xea\x1d\x91\x17\xcb\x9e\xb4\x9b"\xef\xc3\xdf(\xb3'
Signature: 4950435367586149304a63647045735a324c6d2b626946767047515567726b54484934727778517357556d5441462f62583167386c435555717877693046636d585168372b6e317957746a54766b54517358397848736f3d
Signature valid? True


In [14]:
# verify the handmade message from the top
# print(f'Private Key BAse58 Encoded: {base58.b58encode(private_key_raw + b"\x01")}')
print(f'Private key not encoded: {private_key_raw.hex() + "01"}')
# bitccoin_private_key = CBitcoinSecret.from_secret_bytes(private_key_raw)
bitccoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_raw.hex() + "01"), 0)
bitccoin_private_key_bytes = bitccoin_private_key

# print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()[:-2]}")
print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()}")

# bitcoin_public_key_bytes = bitccoin_private_key_bytes.pub
# print(f'Public key base58 Encoded: {base58.b58encode(private_key.get_verifying_key().to_string('compressed'))}')
print(f'bitcoin public not encoded: {private_key.get_verifying_key().to_string('compressed').hex()}')

def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
# bitcoin_public_key_bytes = compress_public_key(private_key_raw.get_verifying_key().to_string())
bitcoin_public_key_bytes = private_key.get_verifying_key().to_string('compressed')
print(f"Bitcoin public key: {bitcoin_public_key_bytes.hex()}")

print(f"Bitcoin address from top: {bitcoin_address}")
bitcoin_address_bytes = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(bitcoin_public_key_bytes.hex()))
print(f"Bitcoin address from pubkey: {bitcoin_address_bytes}")

# bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(bytes.fromhex(bitcoin_public_key_bytes.hex()), 0)
# print(f"Bitcoin address: {bitcoin_address_bytes}")


Private key not encoded: 974fae2d62795bf3fdb658f21bcd48c241731984313765d5f389245537b17f4e01
Bitcoin private key: 974fae2d62795bf3fdb658f21bcd48c241731984313765d5f389245537b17f4e01
bitcoin public not encoded: 02b2465d9ebc12e4c856a1347b4df66114eaa717aee3b388e41d520531c851dc13
Bitcoin public key: 02b2465d9ebc12e4c856a1347b4df66114eaa717aee3b388e41d520531c851dc13
Bitcoin address from top: 1JgECVF9zHQKUBRRD9njbFurhCCBXRi9MR
Bitcoin address from pubkey: 1JgECVF9zHQKUBRRD9njbFurhCCBXRi9MR


In [15]:

bitcoin_message_bytes = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_message_bytes.magic} {bitcoin_message_bytes.message} {bitcoin_message_bytes.GetHash().hex()}")

print(f'signature from top: {signature.hex()}')
# print(f'signature from top b64: {base64.b64encode(signature)}')
# signature_bytes = signature.__bytes__()
signature_bytes_b64 = base64.b64encode(signature) 
print(f"Signature from top b64: {signature_bytes_b64}")

#sign message using bitcoin library
signature_bytes = SignMessage(bitccoin_private_key_bytes, bitcoin_message_bytes)
print(f"Signature bytes: {signature_bytes}")

# print(f'signature_bytes_hex: {base64.b64encode(signature_bytes).hex()}')
print(f'signature bytes decoded: {base64.b64decode(signature_bytes).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_public_key_bytes, bitcoin_message_bytes, signature_bytes)}")

Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
signature from top: 00c7d170e3f82f315a18ca9357b8f42866fb12ec65a9c932932b10714182a85465e4ec9c3ed7f167ae20caa4f444390344a3fe701334f284b1a13f30167cd2de
Signature from top b64: b'AMfRcOP4LzFaGMqTV7j0KGb7EuxlqckykysQcUGCqFRl5OycPtfxZ64gyqT0RDkDRKP+cBM08oSxoT8wFnzS3g=='
Signature bytes: b'H6ZmZOvvcdUY2Vg+ag3BCgOBwzJ35m+mrjRDiypvx2cHOAg0wQpNr2mDL2ACr6zbHnDlGjdO1/+jpct2cqZqFZE='
signature bytes decoded: 1fa66664ebef71d518d9583e6a0dc10a0381c33277e66fa6ae34438b2a6fc76707380834c10a4daf69832f6002afacdb1e70e51a374ed7ffa3a5cb7672a66a1591
Signature valid? False
